In [1]:
import cv2
import numpy as np
from numpy import shape
import matplotlib.pyplot as plt 
from image_extractor import feature_extract

# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Agglutinated/image (1).JPG'
# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Brittle/image (1).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Purple/image (10).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_PartiallyPurple/image (1).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Brown/image (1).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_White/image (72).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Slaty/image (71).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Slaty/image (1).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Moldered/image (7).JPG'
image_dir = 'D:/Thesis_data/mlp_data/testing_img/Agglutinated/image(1).JPG'

plt.rcParams['figure.dpi'] = 102.46 

In [2]:
extractor = feature_extract()
image_bgr = cv2.imread(image_dir)
extractor.extract(image_bgr)


In [3]:
((extractor.structure))

array([ 1.49266517e+02,  2.52681066e+02,  1.10409590e+01,  1.22663280e+01,
        1.04408598e+02,  2.03506183e+02,  4.53195658e+00,  1.60173819e+01,
        1.78750015e+02,  2.12625023e+02,  3.80131531e+00,  1.68823128e+01,
        1.13697014e+02,  2.14307721e+02,  1.18149864e+01,  1.48683415e+01,
        1.39815647e+02,  1.81450989e+02,  2.16730688e+01,  1.00536098e+01,
        1.96285191e+02,  2.43936705e+02,  3.02027960e+01,  1.81661765e+01,
        6.52676722e+01,  8.10682016e+01,  9.67119363e+00,  9.10780447e+00,
        4.63418840e+01,  1.06427144e+02,  1.86441593e+00,  9.83360320e+00,
        6.10139597e+01,  8.69011396e+01,  9.21954326e-01,  5.59799070e+00,
        6.35713449e+01,  9.92420117e+01,  1.01742529e+01,  1.12923116e+01,
        1.04645840e+02,  8.78566589e+01,  1.54918444e+01,  4.50660801e+00,
        7.01146921e+01,  1.05814186e+02,  2.67825725e+01,  1.89863179e+01,
       -4.22347943e-01,  4.41262906e-01,  9.77450965e-01,  7.82186487e-01,
        1.19199746e-01,  

In [4]:
print(extractor.n2)

28


In [5]:
shape(extractor.mold)

(56,)

In [6]:
print(extractor.overall_geometry)

[454, 303, 94436.0, 1328.1950801610947, 272.5821228027344, 445.3172607421875, 1.7058068216928317, 98237.5, 231.60321044921875, 0.9613029647537855, 0.686497724662334, 4.47213595499958]


In [7]:
(extractor.overall_rgb_stat)

array([ 8.3918350e+01,  7.2358871e+01,  4.1041512e+01,  5.8199394e+01,
        5.1120045e+01,  3.1164457e+01, -5.4040462e-01, -4.3952909e-01,
       -7.2695855e-03, -1.2936316e+00, -1.2845954e+00, -8.8336498e-01],
      dtype=float32)

In [8]:
extractor.overall_hsv_stat

array([15.336779  , 91.473236  , 84.335815  , 11.306523  , 63.981968  ,
       58.352917  ,  0.34514114, -0.5105055 , -0.55277675,  6.6657534 ,
       -1.3249129 , -1.291766  ], dtype=float32)

In [9]:
shape(extractor.glcm_correlation)

(12,)

In [10]:
shape(extractor.glcm_dissimilarity)

(12,)

In [11]:
shape(extractor.grid_stat)

(192,)

In [12]:
extractor.glcm_grid

array([2.65065297, 2.67841275, 2.54449127, 2.70903583, 2.87250384,
       2.80968106, 2.73295606, 2.85404694, 3.05223165, 2.9239137 ,
       2.86981591, 2.97673082, 0.61927917, 0.61342141, 0.64373994,
       0.60552661, 0.56663021, 0.5818686 , 0.59864852, 0.56993043,
       0.52445018, 0.55450556, 0.56610166, 0.54011978, 2.65065297,
       2.67841275, 2.54449127, 2.70903583, 2.87250384, 2.80968106,
       2.73295606, 2.85404694, 3.05223165, 2.9239137 , 2.86981591,
       2.97673082, 0.61927917, 0.61342141, 0.64373994, 0.60552661,
       0.56663021, 0.5818686 , 0.59864852, 0.56993043, 0.52445018,
       0.55450556, 0.56610166, 0.54011978, 2.65065297, 2.67841275,
       2.54449127, 2.70903583, 2.87250384, 2.80968106, 2.73295606,
       2.85404694, 3.05223165, 2.9239137 , 2.86981591, 2.97673082,
       0.61927917, 0.61342141, 0.64373994, 0.60552661, 0.56663021,
       0.5818686 , 0.59864852, 0.56993043, 0.52445018, 0.55450556,
       0.56610166, 0.54011978, 2.65065297, 2.67841275, 2.54449

In [13]:
shape(extractor.glcm_grid)

(384,)